In [1]:
pip install streamlit transformers torch


Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install sentencepiece



  Obtaining dependency information for sentencepiece from https://files.pythonhosted.org/packages/a2/f6/587c62fd21fc988555b85351f50bbde43a51524caafd63bc69240ded14fd/sentencepiece-0.2.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
    -------------------------------------- 20.5/991.5 kB 330.3 kB/s eta 0:00:03
   -- ------------------------------------ 51.2/991.5 kB 525.1 kB/s eta 0:00:02
   --- ----------------------------------- 81.9/991.5 kB 573.4 kB/s eta 0:00:02
   --- ---------------------------------- 102.4/991.5 kB 590.8 kB/s eta 0:00:02
   ----- -------------------------------- 153.6/991.5 kB 766.6 kB/s eta 0:00:02
   --------- ---------------------------- 245.8/991.5 kB 942.1 kB/s eta 0:00:01
   --------- ---------------------------- 245.8/991.5 kB 942.1 kB/s eta 0:00:01
   -------------- ------------------------- 368.6/991.5 kB 1.1 MB/s eta 0:00:

In [7]:
pip install tokenizers


Note: you may need to restart the kernel to use updated packages.


In [3]:
import streamlit as st
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# Load pretrained ABSA model
@st.cache_resource
def load_model():
    model_name = "yangheng/deberta-v3-base-absa-v1.1"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    return tokenizer, model

tokenizer, model = load_model()

# Define sentiment labels
labels = ["negative", "neutral", "positive"]

# Aspect-based prediction function
def predict_aspect_sentiment(review, aspect):
    inputs = tokenizer(f"{review} [SEP] {aspect}", return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    probs = F.softmax(outputs.logits, dim=-1)
    sentiment = labels[torch.argmax(probs)]
    return sentiment

# Streamlit UI
st.title("Aspect-Based Sentiment Analysis App")
st.write("Enter a product review and specify aspects you want to analyze.")

review = st.text_area("Product Review", "The screen is beautiful but the battery life is short.")
aspects_input = st.text_input("Aspects to analyze (comma-separated)", "screen, battery")

if st.button("Analyze"):
    aspects = [a.strip() for a in aspects_input.split(",")]
    st.subheader("Results")
    results = {}
    for aspect in aspects:
        sentiment = predict_aspect_sentiment(review, aspect)
        results[aspect] = sentiment
        st.write(f"**{aspect}**: {sentiment.capitalize()}")


2025-04-20 18:13:35.383 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 18:13:35.386 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 18:13:35.389 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 18:13:35.896 Thread 'Thread-7': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 18:13:35.896 Thread 'Thread-7': missing ScriptRunContext! This warning can be ignored when running in bare mode.
C:\Users\simra\anaconda3\Lib\site-packages\transformers\convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version 

2025-04-20 18:18:35.186 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 18:18:35.186 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 18:18:35.188 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 18:18:35.190 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 18:18:35.190 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 18:18:35.191 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 18:18:35.191 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 18:18:35.191 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [5]:
import pandas as pd
import matplotlib.pyplot as plt

# Add upload option
st.sidebar.header("Upload Review CSV")
uploaded_file = st.sidebar.file_uploader("Upload a CSV with a 'review' column", type=["csv"])

st.title("Aspect-Based Sentiment Analysis App")

if uploaded_file:
    df = pd.read_csv(uploaded_file)
    st.write("Sample of uploaded data:")
    st.dataframe(df.head())
    aspects_input = st.text_input("Aspects to analyze (comma-separated)", "battery, screen, camera")

    if st.button("Analyze Reviews"):
        aspects = [a.strip() for a in aspects_input.split(",")]
        all_results = []

        for idx, row in df.iterrows():
            review = row["review"]
            row_result = {"review": review}
            for aspect in aspects:
                sentiment = predict_aspect_sentiment(review, aspect)
                row_result[aspect] = sentiment
            all_results.append(row_result)

        result_df = pd.DataFrame(all_results)
        st.write("📝 Analysis Results:")
        st.dataframe(result_df)

        # Save results
        csv = result_df.to_csv(index=False).encode('utf-8')
        st.download_button("📥 Download Results CSV", csv, "absa_results.csv", "text/csv")

        # Sentiment distribution bar plot
        for aspect in aspects:
            st.subheader(f"📊 Sentiment Distribution for '{aspect}'")
            sentiment_counts = result_df[aspect].value_counts()
            fig, ax = plt.subplots()
            sentiment_counts.plot(kind='bar', ax=ax, color=["red", "gray", "green"])
            ax.set_ylabel("Count")
            st.pyplot(fig)
else:
    st.write("Or enter a single review for quick analysis.")
    review = st.text_area("Review Text")
    aspects_input = st.text_input("Aspects to analyze (comma-separated)", "battery, screen")
    if st.button("Analyze Single Review"):
        aspects = [a.strip() for a in aspects_input.split(",")]
        for aspect in aspects:
            sentiment = predict_aspect_sentiment(review, aspect)
            st.write(f"**{aspect}**: {sentiment.capitalize()}")


2025-04-20 18:24:20.615 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 18:24:20.617 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 18:24:20.623 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 18:24:20.625 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 18:24:20.626 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 18:24:20.629 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 18:24:20.631 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 18:24:20.635 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar